# gevent and greenlet

DesertPy 2015-09-23

Ian Preston (ianpreston.io)

## gevent = greenlet + libev

* cooperative multitasking (greenlet)
* event loop (libev)

## greenlet = lightweight, cooperative multitasking

* Lightweight: Not OS threads, feel free to spawn up thousands
* Cooperative: Need to explicitly yield. Be sure not to block.
* You can beat the rap, but you can't beat the ~~ride~~ GIL
* Great for I/O, not so great for CPU

In [ ]:
import greenlet

a, b = None, None

def foo():
    print 'Beginning of foo'
    b.switch()
    print 'End of foo'

def bar():
    print 'Beginning of bar'
    a.switch()
    print 'End of bar'

a = greenlet.greenlet(foo)
b = greenlet.greenlet(bar)
a.switch()

## libev

* Event loop networking library
* Think of it as Twisted for C
* Wraps epoll()/kqueue()/select()

In [ ]:
%%bash
pip install gevent

Quick note about python 3.5

In [ ]:
import gevent

def foo():
    print 'Hello from foo!'
    # Yield to the scheduler
    gevent.sleep(0)
    print 'Goodbye from foo'

def bar():
    print 'Hello from bar'
    gevent.sleep(0)
    print 'Goodbye from bar'

f = gevent.spawn(foo)
b = gevent.spawn(bar)
gevent.joinall([f, b]) # wait for `f` and `b` to exit

In [ ]:
import gevent
import random

def foo(prefix):
    for x in xrange(4):
        gevent.sleep(random.uniform(0.0, 0.5))
        print prefix, x

a = gevent.spawn(foo, 'thread a')
b = gevent.spawn(foo, 'thread b')
gevent.joinall([a, b])

In [ ]:
return
import gevent
import requests

def get_desertpy_homepage():
    print 'begin...'
    r = requests.get('https://desertpy.org/')
    r.raise_for_status()
    
    print 'downloaded', len(r.content), 'bytes'

threads = [
    gevent.spawn(get_desertpy_homepage)
    for x in xrange(30)
]
gevent.joinall(threads) # May take a while...

## gevent.monkey.patch_all()

    import gevent.monkey
    gevent.monkey.patch_all()

* Monkey-patches standard library packages
* All I/O is now non-blocking and yields automatically
* Syscalls like `time.sleep` yield to gevent
* Threading, locks, etc are now gevent primitives


## gevent.monkey.patch_all()

* Existing code *usually* just works and doesn't need to be modified
* No new API to deal with

In [ ]:
import gevent.monkey
gevent.monkey.patch_all(thread=False)

In [ ]:
import gevent
import time
import requests

def get_desertpy_homepage():
    print 'begin...'
    r = requests.get('http://desertpy.org/')
    r.raise_for_status()
    
    print 'downloaded', len(r.content), 'bytes'

threads = [
    gevent.spawn(get_desertpy_homepage)
    for x in xrange(5)
]
gevent.joinall(threads)

In [ ]:
import gevent
import time

try:
    with gevent.Timeout(0.5):
        time.sleep(10)
except gevent.Timeout:
    print 'Timed out!'

In [ ]:
import gevent

with gevent.Timeout(0.5):
    x = 10
    while True:
        x += x

* You can still shoot yourself in the foot, `monkey.patch_all()` just makes it harder

In [ ]:
# gevent.spawn_later
import gevent

def foo():
    print 'Hello from foo()'

def bar():
    print 'Hello from bar()'
    
gevent.spawn_later(0.5, foo)
gevent.spawn_later(0.2, bar)
gevent.sleep(1.0)

In [ ]:
# gevent.server
import gevent.server

def on_connection(socket, addr):
    print 'on_connection', socket, addr
    greeting = socket.recv(4096)
    socket.sendall('Server says ' + greeting + '\n')
    socket.close()

server = gevent.server.StreamServer(('0.0.0.0', 4200), on_connection)
server.serve_forever()

In [ ]:
# gevent.signal
import gevent
import signal

def handler():
    print 'Got SIGUSR1!'

gevent.signal(signal.SIGUSR1, handler)

In [ ]:
# Event synchronization primitive
import gevent
import gevent.event

ev = gevent.event.Event()

def foo():
    print 'foo: Sleep 1sec'
    gevent.sleep(1.0)
    print 'foo: Set ev'
    ev.set()

def bar():
    print 'bar: Waiting for ev to be set'
    ev.wait()
    print 'bar: ev was set'

gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])

In [ ]:
# Queue synchronization primitive
import gevent
import gevent.queue
import random

q = gevent.queue.Queue()

def worker(worker_name):
    for item in q:
        print 'Worker', worker_name, 'got item', item
        gevent.sleep(random.uniform(0.0, 0.5))

for x in xrange(10):
    q.put(x)

gevent.joinall([
    gevent.spawn(worker, 'A'),
    gevent.spawn(worker, 'B'),
    gevent.spawn(worker, 'C'),
])

In [ ]:
# Semaphore synchronization primitive :\
import gevent
import gevent.lock

s = gevent.lock.BoundedSemaphore(1)

def foo():
    print 'foo: Acquire'
    s.acquire()
    gevent.sleep(2.0)
    print 'foo: Release'
    s.release()

def bar():
    print 'bar: Try to acquire'
    s.acquire()
    print 'bar: Acquired!'
    s.release()

gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])


In [ ]:
# Thread-local variables
import gevent
import gevent.local

val = gevent.local.local()

def foo():
    val.foo = 10
    print 'val.foo=', val.foo


def bar():
    val.bar = 10
    print 'val.bar=', val.bar
    print 'val.foo=', val.foo

gevent.spawn(foo)
gevent.spawn(bar)

In [ ]:
# Pool type
import gevent
import gevent.pool

def foo():
    gevent.sleep(1000)

pool = gevent.pool.Pool(5)

while not pool.full():
    greenlet = pool.spawn(foo)
    print 'Spawned:', greenlet
    print 'Available:', pool.free_count()

In [ ]:
# Pool.imap()
import gevent
import gevent.pool
import requests

def cat_for_status(status):
    print 'Getting HTTP status', status, '...'
    return requests.get('http://http.cat/' + str(status))

#pool = gevent.pool.Pool(1)
#pool = gevent.pool.Pool(7)
pool = gevent.pool.Pool(2)

responses = pool.imap(cat_for_status, [100, 101, 200, 201, 202, 204, 206, 207])
print list(responses)

In [ ]:
# gevent.backdoor
import gevent.backdoor

x = 10

server = gevent.backdoor.BackdoorServer(
    ('0.0.0.0', 4200),
    locals=globals(),
)
server.serve_forever()

In [ ]:
# gevent.wsgi
import gevent.wsgi
import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
    return 'Hello, world!'

server = gevent.wsgi.WSGIServer(('0.0.0.0', 4200), app)
server.serve_forever()

## gevent

* Provides lightweight multitasking and an event loop
* Not dissimilar to `asyncio`
* Great for many IO-bound applications


## links

* Offical site: http://gevent.org/
* Source: https://github.com/gevent/gevent
* Tutorial: http://sdiehl.github.io/gevent-tutorial/